In [ ]:
# Dependencies

from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
from splinter import Browser

In [ ]:
 # executable_path = {'executable_path': 'chromedriver.exe'}
# browser = Browser('chrome', **executable_path, headless=False)

## NASA Mars News

In [ ]:
news_url = 'https://mars.nasa.gov/news/'
browser.visit(news_url)

In [ ]:
# Create a Beautiful Soup object
news_html = browser.html
news_soup = bs(news_html,'html.parser')


In [ ]:
# Extract title  and paragraph text
news_title = news_soup.find("div",class_="content_title").text
news_para = news_soup.find("div", class_="rollover_description_inner").text
print(f"Title is: {news_title}")
print(f"Paragraph is: {news_para}")

## JPL Mars Space Images - Featured Image

In [ ]:
jurl = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(jurl)

In [ ]:
jhtml = browser.html
print(jhtml)

jpl_soup = bs(jhtml,"html.parser")


In [ ]:
image_url = jpl_soup.find('div',class_='carousel_container').article.footer.a['data-fancybox-href']
print(image_url)
base_link = "https:"+jpl_soup.find('div', class_='jpl_logo').a['href'].rstrip('/')
feature_url = base_link+image_url
featured_image_title = jpl_soup.find('h1', class_="media_feature_title").text.strip()
print(f'Featured image title is:{featured_image_title}')
print(f'Featured image url is:{feature_url}')

## Mars Facts

In [ ]:
# visit space facts and scrap the mars facts table
mars_facts_url = 'https://space-facts.com/mars/'
browser.visit(mars_facts_url)
time.sleep(1)
mars_facts_html = browser.html
mars_facts_soup = BeautifulSoup(mars_facts_html, 'html.parser')

fact_table = mars_facts_soup.find('table', class_='tablepress tablepress-id-mars')
column1 = fact_table.find_all('td', class_='column-1')
column2 = fact_table.find_all('td', class_='column-2')

facets = []
values = []

for row in column1:
    facet = row.text.strip()
    facets.append(facet)
    
for row in column2:
    value = row.text.strip()
    values.append(value)
    
mars_facts = pd.DataFrame({
    "Facet":facets,
    "Value":values
    })

mars_facts_html = mars_facts.to_html(header=False, index=False)
mars_facts

## Mars Hemispheres

In [ ]:
# scrape images of Mars' hemispheres from the USGS site
mars_hemisphere_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
hemi_dicts = []

for i in range(1,9,2):
    hemi_dict = {}
    
    browser.visit(mars_hemisphere_url)
    time.sleep(1)
    hemispheres_html = browser.html
    hemispheres_soup = BeautifulSoup(hemispheres_html, 'html.parser')
    hemi_name_links = hemispheres_soup.find_all('a', class_='product-item')
    hemi_name = hemi_name_links[i].text.strip('Enhanced')
    
    detail_links = browser.find_by_css('a.product-item')
    detail_links[i].click()
    time.sleep(1)
    browser.find_link_by_text('Sample').first.click()
    time.sleep(1)
    browser.windows.current = browser.windows[-1]
    hemi_img_html = browser.html
    browser.windows.current = browser.windows[0]
    browser.windows[-1].close()
    
    hemi_img_soup = BeautifulSoup(hemi_img_html, 'html.parser')
    hemi_img_path = hemi_img_soup.find('img')['src']

    print(hemi_name)
    hemi_dict['title'] = hemi_name.strip()
    
    print(hemi_img_path)
    hemi_dict['img_url'] = hemi_img_path

    hemi_dicts.append(hemi_dict)